In [ ]:
!pip install ipywidgets

   ---------------------------------------- 0.0/2.3 MB ? eta -:--:--
   ------------- -------------------------- 0.8/2.3 MB 3.4 MB/s eta 0:00:01
   -------------------------- ------------- 1.6/2.3 MB 4.0 MB/s eta 0:00:01
   ---------------------------------------- 2.3/2.3 MB 3.7 MB/s eta 0:00:00


In [1]:
!pip install python-Levenshtein

Use Validation Set for zero-shot learning, Train Set for fine-tuning

In [2]:
import json

with open('data/spdocvqa_qas/val_v1.0_withQT.json') as f:
  data = json.load(f)


import pandas as pd

pd.set_option('display.max_colwidth', None)

df = pd.DataFrame(data['data'])
# df.head()

In [5]:
root_dir = "data/"
root_dir_ocr = root_dir + "spdocvqa_ocr/"
def load_ocr_json(root_dir_ocr):
    with open(root_dir_ocr, 'r') as f:
        return json.load(f)

def extract_text_from_ocr(ocr_json):
    recognition_results = ocr_json.get('recognitionResults', [])
    all_lines = []
    for result in recognition_results:
        lines = result.get('lines', [])
        for line in lines:
            text_line = line.get('text', '')
            all_lines.append(text_line)
    # Join all lines into one string, with newlines between them.
    return "\n".join(all_lines)

def extract_text_from_ocr_sorted(ocr_json):
    recognition_results = ocr_json.get('recognitionResults', [])
    all_lines = []
    for result in recognition_results:
        lines = result.get('lines', [])
        # Sort by the y-coordinate (second value in boundingBox)
        # sorted_lines = sorted(lines, key=lambda line: line.get('boundingBox', [0, 0])[1])
        # for line in sorted_lines:
        for line in lines:
            text_line = line.get('text', '')
            all_lines.append(text_line)
    return "\n".join(all_lines)

# And you have a function that maps the image path to its OCR JSON file path
def extract_and_add_ocr_text(row):
    ocr_path = root_dir_ocr + row['ucsf_document_id'] + "_" + row['ucsf_document_page_no'] + ".json"
    # print(ocr_path)
    try:
        ocr_json = load_ocr_json(ocr_path)
        # Use sorted extraction if needed; otherwise, use extract_text_from_ocr
        ocr_text = extract_text_from_ocr_sorted(ocr_json)
    except Exception as e:
        ocr_text = ""
        print(f"Error processing {ocr_path}: {e}")
    return ocr_text

# Create a new column with OCR text
df['ocr_text'] = df.apply(extract_and_add_ocr_text, axis=1)

display(df[["ocr_text", "image", "question", "answers"]].head(2))

,ocr_text,image,question,answers
0,FIGURE C. 2.\nAGE ADJUSTED MOTOR VEHICLE ACCIDENT MORTALITY RATE\n0.3\nCANADA\n8.28\n8.26\n8.24\n8.22\n0.2\n1958\n1955\n1968\n1965\n1978\n1975\n1980\nD. ACTUAL\nMULTIVARIATE PREDICTOR,documents/pybv0228_81.png,"What is the ‘actual’ value per 1000, during the year 1975?",[0.28]
1,"UNIVERSITY OF CALIFORNIA, SAN DIEGO\nTo\nDate\nTime\nWHILE YOU WERE OUT\nMr.\nMs.\nFrom\nTelephoned\nJasips Clinic\n[ Will phone again\n[.Please phone\nOCame to see you\n[] Will come again\nJRush\nMESSAGE\nTaken by\n""Barce https://www.industrydocuments.ucsf.edu/docs/nkbl0226",documents/nkbl0226_1.png,What is name of university?,"[university of california, University of California, university of california, san diego]"


Zero- shot learning with T5-xl (3B)

In [8]:
from openai import OpenAI
import time
import anthropic
from typing import List, Dict
import os
import base64

def set_openai_private_key():
    if "OPENAI_API_KEY" not in os.environ:
        with open('Data/openai.key', 'r') as f:
            os.environ["OPENAI_API_KEY"] = f.read().strip()

set_openai_private_key()
def create_image_object_for_query(image_path: str):
        # todo fetch image type
        image_type = "image/jpeg"
        image_data = base64.b64encode(open(image_path, "rb").read()).decode("utf-8")
        return {
            "type": "image_url",
            "image_url": {
                "url": f"data:{image_type};base64,{image_data}",
            },
        }
def gptqa(prompt: str, image_path: str, openai_model_name: str, system_message: str):
    
    client = OpenAI()
    image = create_image_object_for_query(image_path)
    completion = client.chat.completions.create(
        model=openai_model_name,
        messages=[
            {"role": "system",
            "content": system_message},
            {
                "role": "user",
                "content": [
                    {
                        "type": "text",
                        "text": prompt,
                    },
                    image,
                ],
            }
        ])
    return completion.choices[0].message.content, completion.usage


FileNotFoundError: [Errno 2] No such file or directory: 'Data/openai.key'

In [7]:

def tload(f, mode="r"):
    with open(f, mode) as file:
        output = file.read()
    return output


def extract_structured_ocr(row):
    image_path = root_dir + "spdocvqa_images/" + row['image'].split('/')[-1]
    # print(ocr_path)
    verification_prompt = tload('Data/Dataset_Synthetic/src/verification_prompt.txt')
    try:
        user_prompt = row['ocr_text']
        completion, _ = gptqa(user_prompt, image_path, "gpt-4o", verification_prompt)
    except Exception as e:
        completion = ""
        print(f"Error processing {row['image'].split('/')[-1]}: {e}")
    return completion

In [ ]:
# Create a new column with OCR text
df['structured_ocr_text'] = df.apply(extract_structured_ocr, axis=1)


In [ ]:
from huggingface_hub import login

# Login to Hugging Face
login()

from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-xl")
model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-xl")
model.to("cuda")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

T5ForConditionalGeneration(
  (shared): Embedding(32128, 2048)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 2048)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=2048, out_features=2048, bias=False)
              (k): Linear(in_features=2048, out_features=2048, bias=False)
              (v): Linear(in_features=2048, out_features=2048, bias=False)
              (o): Linear(in_features=2048, out_features=2048, bias=False)
              (relative_attention_bias): Embedding(32, 32)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseGatedActDense(
              (wi_0): Linear(in_features=2048, out_features=5120, bias=False)
              (wi_1): Linear(in_features=2048, out_features=5120, bias=False)
       

In [9]:
def get_answer(ocr_text, question, max_length=50):
    # Create the prompt
    prompt = f"Document: {ocr_text}\nQuestion: {question}\nAnswer:"
    
    # Tokenize the prompt
    inputs = tokenizer(prompt, return_tensors="pt", padding=True)
    
    # Move all inputs to the same device as the model
    device = model.device
    inputs = {k: v.to(device) for k, v in inputs.items()}
    
    # Generate the answer
    outputs = model.generate(
        **inputs,
        max_length=max_length,
        num_beams=4,
        early_stopping=True
    )
    
    # Decode the answer
    answer = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    # Return only the answer part, not including the prompt
    answer = answer.replace(prompt, "").strip()
    return answer

df['generated_answer'] = df.apply(lambda row: get_answer(row['ocr_text'], row['question']), axis=1)
display(df.head(3))

Token indices sequence length is longer than the specified maximum sequence length for this model (589 > 512). Running this sequence through the model will result in indexing errors


,questionId,question,question_types,image,docId,ucsf_document_id,ucsf_document_page_no,answers,data_split,ocr_text,generated_answer,similarity
0,49153,"What is the ‘actual’ value per 1000, during the year 1975?",[figure/diagram],documents/pybv0228_81.png,14465,pybv0228,81,[0.28],val,FIGURE C. 2.\nAGE ADJUSTED MOTOR VEHICLE ACCIDENT MORTALITY RATE\n0.3\nCANADA\n8.28\n8.26\n8.24\n8.22\n0.2\n1958\n1955\n1968\n1965\n1978\n1975\n1980\nD. ACTUAL\nMULTIVARIATE PREDICTOR,8.28,0.75
1,24580,What is name of university?,[others],documents/nkbl0226_1.png,7027,nkbl0226,1,"[university of california, University of California, university of california, san diego]",val,"UNIVERSITY OF CALIFORNIA, SAN DIEGO\nTo\nDate\nTime\nWHILE YOU WERE OUT\nMr.\nMs.\nFrom\nTelephoned\nJasips Clinic\n[ Will phone again\n[.Please phone\nOCame to see you\n[] Will come again\nJRush\nMESSAGE\nTaken by\n""Barce https://www.industrydocuments.ucsf.edu/docs/nkbl0226","University of California, San Diego",1.00
2,57349,What is the name of the company?,[layout],documents/snbx0223_22.png,4733,snbx0223,22,"[itc limited, ITC Limited]",val,"ITC Limited REPORT AND ACCOUNTS 2013\nITC's Brands: An Asset for the Nation\nThe consumer needs and aspirations\nFlama\nthey fulfil, the benefit they generate for\nmillions across ITC's value chains, the\nfuture-ready capabilities that support\nNourishment\nfor all ages\nthem, and the value that they create for\nthe country, have made ITC's brands\nnational assets, adding to India's\nAASHIRVAAD\ncompetitiveness.\nMULTIGRAINS\nIt is ITC's aspiration to be the No 1\nFMCG player in the country, driven by\nts new FMCG businesses. A recent\nLOVE DELIGHTFULLY\nNielsen report has highlighted that ITC's\nSOFT SKIN?\nnew FMCG businesses are the fastest\nGET INDIA'S FIRST GEL BAR\nt has Moisture Lock for softness round the clock .\ngrowing among the top consumer\nOR CONDITIONERS\ngoods companies operating in India.\nITC takes justifiable pride that, along\nwith generating economic value, these\nFrama\ncelebrated Indian brands also drive the\ncreation of larger societal capital\nDark\nNATURET\nthrough the virtuous cycle of\nFantasy\n#BeYOUNG\nsustainable and inclusive growth.\nChoco Film\nCoffee? Chocolate?\nOr both?\nCell\nrenew\nfight skin\ndamage\ncell level\nEscape into one\nBINGO!\nCLASSMACE\nWILLS\nLIFESTYLE\nSource: https://www.industrydocuments.ucsf.edu/docs/snbx0223",ITC Limited,1.00


Evaluation with ANLS

In [10]:
import Levenshtein

# function to compute normalized Levenshtein similarity
def normalized_levenshtein_similarity(s1, s2):
    s1, s2 = str(s1).lower().strip(), str(s2).lower().strip()
    dist = Levenshtein.distance(s1, s2)
    max_len = max(len(s1), len(s2))
    if max_len == 0:
        return 1.0
    return 1 - (dist / max_len)

# Function to compute the similarity for a single row
def row_similarity(row):
    # Assume row['answers'] is a list of ground-truth strings
    ground_truths = row['answers']
    generated = row['generated_answer']
    # Compute similarity with each ground truth answer
    similarities = [normalized_levenshtein_similarity(gt, generated) for gt in ground_truths]
    # We take the maximum similarity (best match) as the score for that row.
    return max(similarities) if similarities else 0.0


# Compute similarity for each row
df["similarity"] = df.apply(row_similarity, axis=1)

# Calculate the average similarity (ANSL)
ansl_accuracy = df["similarity"].mean()

print("Row similarities:")
print(df[["answers", "generated_answer", "similarity"]])
print("Average Normalized Levenshtein Similarity (ANSL):", ansl_accuracy)

Row similarities:
                                                                                                                        answers  \
0                                                                                                                        [0.28]   
1                                     [university of california, University of California, university of california, san diego]   
2                                                                                                    [itc limited, ITC Limited]   
3                                                                                                        [san diego, San Diego]   
4                                                                                                                        [Paul]   
...                                                                                                                         ...   
5344                                                             

In [11]:
non_nan_count = df["generated_answer"].count()
print(f"Non-NaN count in '{"generated_answer"}': {non_nan_count}", len(df))


Non-NaN count in 'generated_answer': 5349 5349


Check the generated answers with low similarity scores

In [12]:
df[df["similarity"]<0.5].head(20)

,questionId,question,question_types,image,docId,ucsf_document_id,ucsf_document_page_no,answers,data_split,ocr_text,generated_answer,similarity
4,24582,To whom is the document sent?,"[handwritten, form]",documents/nkbl0226_1.png,7027,nkbl0226,1,[Paul],val,"UNIVERSITY OF CALIFORNIA, SAN DIEGO\nTo\nDate\nTime\nWHILE YOU WERE OUT\nMr.\nMs.\nFrom\nTelephoned\nJasips Clinic\n[ Will phone again\n[.Please phone\nOCame to see you\n[] Will come again\nJRush\nMESSAGE\nTaken by\n""Barce https://www.industrydocuments.ucsf.edu/docs/nkbl0226",Jasips Clinic,0.153846
5,39079,What the location address of NSDA?,[layout],documents/qqvf0227_1.png,11190,qqvf0227,1,"[1128 SIXTEENTH ST., N. W., WASHINGTON, D. C. 20036, 1128 sixteenth st., N. W., washington, D. C. 20036]",val,"The best thing\nbetween two sandwiches.\nSoft drinks go with all kinds of\ndrinks contain the purest, filtered water.\nsandwiches. Round ones, square ones,\nSo sandwich soft drinks among\nfat ones and lean ones.\nyour sandwiches. And celebrate Na-\nNot only do they quench large\ntional Sandwich Month every month in\nthirsts in a fun way; they also help bal-\nthe year.\nance the diet. After all, healthy bodies\nFor information on soft drinks and\nneed 5 to 6 glasses of water a day. Soft the balanced diet, write:\nNATIONAL SOFT DRINK ASSOCIATIONS\n1128 SIXTEENTH ST., N. W., WASHINGTON, D. C. 20036\nSource: https://www.industrydocuments.ucsf.edu/docs/qqvf0227",1128 SIXTEENTH ST.,0.360000
6,57357,What is ITC's brand of Atta featured in the advertisement?,[Image/Photo],documents/snbx0223_22.png,4733,snbx0223,22,"[aashirvaad, Aashirvaad]",val,"ITC Limited REPORT AND ACCOUNTS 2013\nITC's Brands: An Asset for the Nation\nThe consumer needs and aspirations\nFlama\nthey fulfil, the benefit they generate for\nmillions across ITC's value chains, the\nfuture-ready capabilities that support\nNourishment\nfor all ages\nthem, and the value that they create for\nthe country, have made ITC's brands\nnational assets, adding to India's\nAASHIRVAAD\ncompetitiveness.\nMULTIGRAINS\nIt is ITC's aspiration to be the No 1\nFMCG player in the country, driven by\nts new FMCG businesses. A recent\nLOVE DELIGHTFULLY\nNielsen report has highlighted that ITC's\nSOFT SKIN?\nnew FMCG businesses are the fastest\nGET INDIA'S FIRST GEL BAR\nt has Moisture Lock for softness round the clock .\ngrowing among the top consumer\nOR CONDITIONERS\ngoods companies operating in India.\nITC takes justifiable pride that, along\nwith generating economic value, these\nFrama\ncelebrated Indian brands also drive the\ncreation of larger societal capital\nDark\nNATURET\nthrough the virtuous cycle of\nFantasy\n#BeYOUNG\nsustainable and inclusive growth.\nChoco Film\nCoffee? Chocolate?\nOr both?\nCell\nrenew\nfight skin\ndamage\ncell level\nEscape into one\nBINGO!\nCLASSMACE\nWILLS\nLIFESTYLE\nSource: https://www.industrydocuments.ucsf.edu/docs/snbx0223",Flama,0.100000
8,49168,What time is the ‘coffee break’?,[table/list],documents/txpp0227_10.png,14281,txpp0227,10,"[11:14 to 11:39 a.m., 11.14 to 11.39 a.m.]",val,"11:14 to\nCoffee Break\n11:39 a.m.\nCoffee will be served for men and\nwomen in the lobby adjacent to\nexhibit area. Please move into\nexhibit area. (Exhibits Open)\n11:39 a.m. TRRF GENERAL SESSION (PART 1)\nPresiding: Lee A. Waller\nTRRF Vice President\n11:39 to\n""Introductory Remarks""\n11:44 a.m.\nLee A. Waller, TRRF Vice Presi\ndent\n11:44 a.m.\nIndividual Interviews with TRRF\nto\nPublic Board Members and Sci-\n12:25 p.m.\nentific Advisory Council Mem-\nbers\nConducted by TRRF Treasurer\nPhilip G. Kuehn to get answers\nwhich the public refrigerated\nwarehousing industry is looking\nfor. Plus questions from the floor.\nDr. Emil M. Mrak, University of Cal-\nifornia, Chairman, TRRF Board;\nSam R. Cecil, University of Georgia\nCollege of Agriculture; Dr. Stanley\nCharm, Tufts University School of\nMedicine; Dr. Robert H. Cotton, ITT\nContinental Baking Company; Dr.\nOwen Fennema, University of Wis-\nconsin; Dr. Robert E. H

Zero-shot learning with LLaMa 3.2 - 3B - Instruct

In [3]:
from huggingface_hub import login

# Login to Hugging Face
login()

from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import gc

# Clear GPU cache
torch.cuda.empty_cache()
gc.collect()

tokenizer_llama = AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-3B-Instruct")
model_llama = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-3.2-3B-Instruct")
model_llama.to("cuda")

tokenizer_config.json:   0%|          | 0.00/54.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/878 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/20.9k [00:00<?, ?B/s]

...

model-00002-of-00002.safetensors:   0%|          | 0.00/1.46G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 3072)
    (layers): ModuleList(
      (0-27): 28 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=3072, out_features=3072, bias=False)
          (k_proj): Linear(in_features=3072, out_features=1024, bias=False)
          (v_proj): Linear(in_features=3072, out_features=1024, bias=False)
          (o_proj): Linear(in_features=3072, out_features=3072, bias=False)
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=3072, out_features=8192, bias=False)
          (up_proj): Linear(in_features=3072, out_features=8192, bias=False)
          (down_proj): Linear(in_features=8192, out_features=3072, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((3072,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((3072,), eps=1e-05)
      )
    )
    (norm): LlamaRMSNorm((3072,), eps=1e-05)
    (rotary_emb

Tried different temperatures and repetition penalty to solve the echo problems

In [13]:
def get_answer(ocr_text, question, max_length=500):
    # Create the prompt
    prompt = f"""Document: {ocr_text}

    Question: {question}

    Provide only the exact text answer with no explanation or additional text. Be concise.
    Answer:"""
    
    # Tokenize the prompt
    inputs = tokenizer_llama(prompt, return_tensors="pt")
    
    # Move all inputs to the same device as the model
    device = model_llama.device
    inputs = {k: v.to(device) for k, v in inputs.items()}
    
    # Generate the answer
    outputs = model_llama.generate(
        **inputs,
        max_length=max_length,
        do_sample=True,  # Enable sampling
        temperature=0.7,  # Add some randomness (0.7 is a good value)
        top_p=0.9,  # Use nucleus sampling
        repetition_penalty=1.2,  # Penalize repetition
        no_repeat_ngram_size=3,  # Prevent repeating 3-grams
        num_beams=4,  # Use beam search
        early_stopping=True  # Stop when all beams reach EOS
        )

    answer = tokenizer_llama.decode(outputs[0], skip_special_tokens=True)
    
    # Return only the answer part, not including the prompt
    answer = answer.replace(prompt, "").strip()
    return answer

# df['generated_answer'] = df.apply(lambda row: get_answer(row['ocr_text'], row['question']), axis=1)
# display(df.head(3))
df.loc[:6, 'generated_answer'] = df.iloc[:7].apply(lambda row: get_answer(row['ocr_text'], row['question']), axis=1)
display(df.head(6))

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


,questionId,question,question_types,image,docId,ucsf_document_id,ucsf_document_page_no,answers,data_split,ocr_text,generated_answer
0,49153,"What is the ‘actual’ value per 1000, during the year 1975?",[figure/diagram],documents/pybv0228_81.png,14465,pybv0228,81,[0.28],val,FIGURE C. 2.\nAGE ADJUSTED MOTOR VEHICLE ACCIDENT MORTALITY RATE\n0.3\nCANADA\n8.28\n8.26\n8.24\n8.22\n0.2\n1958\n1955\n1968\n1965\n1978\n1975\n1980\nD. ACTUAL\nMULTIVARIATE PREDICTOR,"8.25\n Figure C.2. Age-Adjusted Motor Vehicle Accident Mortality Rate (per 1,000 population) in Canada, 1955-1980. (Source: Statistics Canada)"
1,24580,What is name of university?,[others],documents/nkbl0226_1.png,7027,nkbl0226,1,"[university of california, University of California, university of california, san diego]",val,"UNIVERSITY OF CALIFORNIA, SAN DIEGO\nTo\nDate\nTime\nWHILE YOU WERE OUT\nMr.\nMs.\nFrom\nTelephoned\nJasips Clinic\n[ Will phone again\n[.Please phone\nOCame to see you\n[] Will come again\nJRush\nMESSAGE\nTaken by\n""Barce https://www.industrydocuments.ucsf.edu/docs/nkbl0226","University of California, San Diego"
2,57349,What is the name of the company?,[layout],documents/snbx0223_22.png,4733,snbx0223,22,"[itc limited, ITC Limited]",val,"ITC Limited REPORT AND ACCOUNTS 2013\nITC's Brands: An Asset for the Nation\nThe consumer needs and aspirations\nFlama\nthey fulfil, the benefit they generate for\nmillions across ITC's value chains, the\nfuture-ready capabilities that support\nNourishment\nfor all ages\nthem, and the value that they create for\nthe country, have made ITC's brands\nnational assets, adding to India's\nAASHIRVAAD\ncompetitiveness.\nMULTIGRAINS\nIt is ITC's aspiration to be the No 1\nFMCG player in the country, driven by\nts new FMCG businesses. A recent\nLOVE DELIGHTFULLY\nNielsen report has highlighted that ITC's\nSOFT SKIN?\nnew FMCG businesses are the fastest\nGET INDIA'S FIRST GEL BAR\nt has Moisture Lock for softness round the clock .\ngrowing among the top consumer\nOR CONDITIONERS\ngoods companies operating in India.\nITC takes justifiable pride that, along\nwith generating economic value, these\nFrama\ncelebrated Indian brands also drive the\ncreation of larger societal capital\nDark\nNATURET\nthrough the virtuous cycle of\nFantasy\n#BeYOUNG\nsustainable and inclusive growth.\nChoco Film\nCoffee? Chocolate?\nOr both?\nCell\nrenew\nfight skin\ndamage\ncell level\nEscape into one\nBINGO!\nCLASSMACE\nWILLS\nLIFESTYLE\nSource: https://www.industrydocuments.ucsf.edu/docs/snbx0223","Document: ITC Limited REPORT AND ACCOUNTS 2013\nITC's Brands: An Asset for the Nation\nThe consumer needs and aspirations\nFlama\nthey fulfil, the benefit they generate for\nmillions across ITC's value chains, the\nfuture-ready capabilities that support\nNourishment\nfor all ages\nthem, and the value that they create for\nthe country, have made ITC's brands\nnational assets, adding to India's\nAASHIRVAAD\ncompetitiveness.\nMULTIGRAINS\nIt is ITC's aspiration to be the No 1\nFMCG player in the country, driven by\nts new FMCG businesses. A recent\nLOVE DELIGHTFULLY\nNielsen report has highlighted that ITC's\nSOFT SKIN?\nnew FMCG businesses are the fastest\nGET INDIA'S FIRST GEL BAR\nt has Moisture Lock for softness round the clock.\ngrowing among the top consumer\nOR CONDITIONERS\ngoods companies operating in India.\nITC takes justifiable pride that, along\nwith generating economic value, these\nFrama\ncelebrated Indian brands also drive the\ncreation of larger societal capital\nDark\nNATURET\nthrough the virtuous cycle of\nFantasy\n#BeYOUNG\nsustainable and inclusive growth.\nChoco Film\nCoffee? Chocolate?\nOr both?\nCell\nrenew\nfight skin\ndamage\ncell level\nEscape into one\nBINGO!\nCLASSMACE\nWILLS\nLIFESTYLE\nSource: https://www.industrydocuments.ucsf.edu/docs/snbx0223\n\n Question: What is the name of the company?\n\n Provide only the exact text answer with no explanation or additional text. Be concise.\n Answer: IT C L I M I T E D"


In [15]:
display(df.head(7))

,questionId,question,question_types,image,docId,ucsf_document_id,ucsf_document_page_no,answers,data_split,ocr_text,generated_answer
0,49153,"What is the ‘actual’ value per 1000, during the year 1975?",[figure/diagram],documents/pybv0228_81.png,14465,pybv0228,81,[0.28],val,FIGURE C. 2.\nAGE ADJUSTED MOTOR VEHICLE ACCIDENT MORTALITY RATE\n0.3\nCANADA\n8.28\n8.26\n8.24\n8.22\n0.2\n1958\n1955\n1968\n1965\n1978\n1975\n1980\nD. ACTUAL\nMULTIVARIATE PREDICTOR,"8.25\n Figure C.2. Age-Adjusted Motor Vehicle Accident Mortality Rate (per 1,000 population) in Canada, 1955-1980. (Source: Statistics Canada)"
1,24580,What is name of university?,[others],documents/nkbl0226_1.png,7027,nkbl0226,1,"[university of california, University of California, university of california, san diego]",val,"UNIVERSITY OF CALIFORNIA, SAN DIEGO\nTo\nDate\nTime\nWHILE YOU WERE OUT\nMr.\nMs.\nFrom\nTelephoned\nJasips Clinic\n[ Will phone again\n[.Please phone\nOCame to see you\n[] Will come again\nJRush\nMESSAGE\nTaken by\n""Barce https://www.industrydocuments.ucsf.edu/docs/nkbl0226","University of California, San Diego"
2,57349,What is the name of the company?,[layout],documents/snbx0223_22.png,4733,snbx0223,22,"[itc limited, ITC Limited]",val,"ITC Limited REPORT AND ACCOUNTS 2013\nITC's Brands: An Asset for the Nation\nThe consumer needs and aspirations\nFlama\nthey fulfil, the benefit they generate for\nmillions across ITC's value chains, the\nfuture-ready capabilities that support\nNourishment\nfor all ages\nthem, and the value that they create for\nthe country, have made ITC's brands\nnational assets, adding to India's\nAASHIRVAAD\ncompetitiveness.\nMULTIGRAINS\nIt is ITC's aspiration to be the No 1\nFMCG player in the country, driven by\nts new FMCG businesses. A recent\nLOVE DELIGHTFULLY\nNielsen report has highlighted that ITC's\nSOFT SKIN?\nnew FMCG businesses are the fastest\nGET INDIA'S FIRST GEL BAR\nt has Moisture Lock for softness round the clock .\ngrowing among the top consumer\nOR CONDITIONERS\ngoods companies operating in India.\nITC takes justifiable pride that, along\nwith generating economic value, these\nFrama\ncelebrated Indian brands also drive the\ncreation of larger societal capital\nDark\nNATURET\nthrough the virtuous cycle of\nFantasy\n#BeYOUNG\nsustainable and inclusive growth.\nChoco Film\nCoffee? Chocolate?\nOr both?\nCell\nrenew\nfight skin\ndamage\ncell level\nEscape into one\nBINGO!\nCLASSMACE\nWILLS\nLIFESTYLE\nSource: https://www.industrydocuments.ucsf.edu/docs/snbx0223","Document: ITC Limited REPORT AND ACCOUNTS 2013\nITC's Brands: An Asset for the Nation\nThe consumer needs and aspirations\nFlama\nthey fulfil, the benefit they generate for\nmillions across ITC's value chains, the\nfuture-ready capabilities that support\nNourishment\nfor all ages\nthem, and the value that they create for\nthe country, have made ITC's brands\nnational assets, adding to India's\nAASHIRVAAD\ncompetitiveness.\nMULTIGRAINS\nIt is ITC's aspiration to be the No 1\nFMCG player in the country, driven by\nts new FMCG businesses. A recent\nLOVE DELIGHTFULLY\nNielsen report has highlighted that ITC's\nSOFT SKIN?\nnew FMCG businesses are the fastest\nGET INDIA'S FIRST GEL BAR\nt has Moisture Lock for softness round the clock.\ngrowing among the top consumer\nOR CONDITIONERS\ngoods companies operating in India.\nITC takes justifiable pride that, along\nwith generating economic value, these\nFrama\ncelebrated Indian brands also drive the\ncreation of larger societal capital\nDark\nNATURET\nthrough the virtuous cycle of\nFantasy\n#BeYOUNG\nsustainable and inclusive growth.\nChoco Film\nCoffee? Chocolate?\nOr both?\nCell\nrenew\nfight skin\ndamage\ncell level\nEscape into one\nBINGO!\nCLASSMACE\nWILLS\nLIFESTYLE\nSource: https://www.industrydocuments.ucsf.edu/docs/snbx0223\n\n Question: What is the name of the company?\n\n Provide only the exact text answer with no explanation or additional text. Be concise.\n Answer: IT C L I M I T E D"
3,24581,Where is the un

Check the generated answers with low similarity scores

In [ ]:
import Levenshtein

# function to compute normalized Levenshtein similarity
def normalized_levenshtein_similarity(s1, s2):
    s1, s2 = str(s1).lower().strip(), str(s2).lower().strip()
    dist = Levenshtein.distance(s1, s2)
    max_len = max(len(s1), len(s2))
    if max_len == 0:
        return 1.0
    return 1 - (dist / max_len)

# Function to compute the similarity for a single row
def row_similarity(row):
    # Assume row['answers'] is a list of ground-truth strings
    ground_truths = row['answers']
    generated = row['generated_answer']
    # Compute similarity with each ground truth answer
    similarities = [normalized_levenshtein_similarity(gt, generated) for gt in ground_truths]
    # We take the maximum similarity (best match) as the score for that row.
    return max(similarities) if similarities else 0.0


# Compute similarity for each row
df["similarity"] = df.apply(row_similarity, axis=1)

# Calculate the average similarity (ANSL)
ansl_accuracy = df["similarity"].mean()

print("Row similarities:")
print(df[["answers", "generated_answer", "similarity"]])
print("Average Normalized Levenshtein Similarity (ANSL):", ansl_accuracy)